In [7]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk import pos_tag
from nltk.stem import PorterStemmer, WordNetLemmatizer
import re
import nltk
import pprint as pp
import db_scripts

In [8]:
def load_stop_words(stop_word_file):
    stop_words = []
    fo = open(stop_word_file)
    for line in fo:
        if line.strip()[0:1] != "#":
            for word in line.split():  # in case more than one per line
                stop_words.append(word)
    fo.close()
    return stop_words

In [ ]:
def load_keywords_from_bucket(keyword_file):
    word_map = dict()
    file = open(keyword_file)
    for line in file:
        key = line.strip(':')

In [9]:
def remove_puncts(text):
    tokenizer = RegexpTokenizer(r'\w+')
    words_no_punct = tokenizer.tokenize(text)
    data = ' '.join(words_no_punct)
    return data

In [10]:
def is_number(s):
    try:
        float(s) if '.' in s else int(s)
        return True
    except ValueError:
        return False

In [11]:
def generate_candidate_keywords(data, stop_words):
    words = [w for w in word_tokenize(
        data) if w.lower() not in stop_words and not is_number(w)]
    return words

In [18]:
def stem_lemmatize(keywords):
    lemmatizer = WordNetLemmatizer()
    stemmer = PorterStemmer()
    tokenizer = RegexpTokenizer(r'\w+')
#     table.column_headers = ["Actual", "Lemmatize", "Stem"]
    x = []
    y = []
    for word in keywords:
        x.append(stemmer.stem(word))
        y.append(lemmatizer.lemmatize(word))
    return x

In [19]:
alda = "Introduction to the problems and techniques for automated discovery of knowledge in databases. Topics include representation, evaluation, and formalization of knowledge for discovery; classification, prediction, clustering, and association methods.Selected applications in commerce, security, and bioinformatics. Students cannot get credit for both CSC 422 and CSC 522."

In [20]:
class Keywords:
    def __init__(self, text):
        self.text = text
        self.stop_words = load_stop_words("FoxStopList.txt")

    def fetch(self):
        candidate = remove_puncts(self.text)
        keywords = generate_candidate_keywords(candidate, self.stop_words)
        # table = stem_lemmatize(keywords)
        st = stem_lemmatize(keywords)
        return keywords, st

In [21]:
l, p = Keywords(alda).fetch()
print(p)

['autom', 'discoveri', 'knowledg', 'databas', 'represent', 'evalu', 'formal', 'knowledg', 'discoveri', 'classif', 'predict', 'cluster', 'associ', 'method', 'applic', 'commerc', 'secur', 'bioinformat']


In [22]:
set(p)

{'applic',
 'associ',
 'autom',
 'bioinformat',
 'classif',
 'cluster',
 'commerc',
 'databas',
 'discoveri',
 'evalu',
 'formal',
 'knowledg',
 'method',
 'predict',
 'represent',
 'secur'}

In [23]:
p = set(p)

In [26]:
from collections import defaultdict
d = defaultdict()
for i in p:
    if i in d:
        d[i].append('CSC522')
    else:
        d[i] = ['CSC522']

In [27]:
d

defaultdict(None,
            {'predict': ['CSC522'],
             'knowledg': ['CSC522'],
             'commerc': ['CSC522'],
             'secur': ['CSC522'],
             'classif': ['CSC522'],
             'formal': ['CSC522'],
             'associ': ['CSC522'],
             'method': ['CSC522'],
             'evalu': ['CSC522'],
             'cluster': ['CSC522'],
             'applic': ['CSC522'],
             'discoveri': ['CSC522'],
             'bioinformat': ['CSC522'],
             'databas': ['CSC522'],
             'represent': ['CSC522'],
             'autom': ['CSC522']})

In [29]:
import pandas as pd
df = pd.DataFrame().from_dict(d)

In [30]:
df

,applic,associ,autom,bioinformat,classif,cluster,commerc,databas,discoveri,evalu,formal,knowledg,method,predict,represent,secur
0,CSC522,CSC522,CSC522,CSC522,CSC522,CSC522,CSC522,CSC522,CSC522,CSC522,CSC522,CSC522,CSC522,CSC522,CSC522,CSC522


In [33]:
d['predict'].append('CSC501')
df = pd.DataFrame().from_dict(d,orient='index').transpose()
df

,predict,knowledg,commerc,secur,classif,formal,associ,method,evalu,cluster,applic,discoveri,bioinformat,databas,represent,autom
0,CSC522,CSC522,CSC522,CSC522,CSC522,CSC522,CSC522,CSC522,CSC522,CSC522,CSC522,CSC522,CSC522,CSC522,CSC522,CSC522
1,CSC501,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,CSC501,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,CSC501,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [37]:
import db_scripts
all_courses = db_scripts.db_fetch_all("wolfpal", "courses")

Authentication Successful.


In [38]:
from collections import defaultdict
d = defaultdict()

In [39]:
for course in all_courses:
    l, p = Keywords(course['desc']).fetch()
    # print(course['desc'])
    p = set(p)
    for i in p:
        if i in d:
            d[i].append(course['branch']+course['number'])
        else:
            d[i] = [(course['branch']+course['number'])]

In [40]:
d

defaultdict(None,
            {'fundament': ['CSC501'],
             'system': ['CSC501', 'CSC520'],
             'process': ['CSC501', 'CSC520'],
             'element': ['CSC501', 'CSC520'],
             'manag': ['CSC501', 'CSC510'],
             'memori': ['CSC501'],
             'schedul': ['CSC501', 'CSC510'],
             'oper': ['CSC501'],
             'relat': ['CSC501', 'CSC541'],
             'distribut': ['CSC501'],
             'coordin': ['CSC501'],
             'issu': ['CSC501'],
             'design': ['CSC501', 'CSC510', 'CSC541'],
             'deadlock': ['CSC501'],
             'test': ['CSC510'],
             'risk': ['CSC510'],
             'code': ['CSC510', 'CSC541'],
             'estim': ['CSC510'],
             'review': ['CSC510'],
             'chang': ['CSC510'],
             'emphasi': ['CSC510'],
             'semest': ['CSC510'],
             'reliabl': ['CSC510'],
             'project': ['CSC510'],
             'softwar': ['CSC510', 'CSC541'],
     

In [41]:
import pandas as pd
df = pd.DataFrame().from_dict(d,orient='index').transpose()

In [42]:
df

,fundament,system,process,element,manag,memori,schedul,oper,relat,distribut,...,AI,natur,knowledg,languag,solv,space,prolog,program,represent,studi
0,CSC501,CSC501,CSC501,CSC501,CSC501,CSC501,CSC501,CSC501,CSC501,CSC501,...,CSC520,CSC520,CSC520,CSC520,CSC520,CSC520,CSC520,CSC520,CSC520,CSC520
1,None,CSC520,CSC520,CSC520,CSC510,None,CSC510,None,CSC541,None,...,None,None,None,None,None,None,None,None,None,None
2,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [43]:
usersays = "I like AI and systems"
p,l = Keywords(usersays).fetch()
print(l)

['AI', 'system']


In [47]:
for i in l:
    if i in d:
        print("Yeh course le: ",d[i])

Yeh course le:  ['CSC520']
Yeh course le:  ['CSC501', 'CSC520']
